In [1]:
import matplotlib.pyplot as plt

In [2]:
def make_graph(point_1, point_2, obstacles):
    plt.scatter(point_1[0], point_1[1], c="blue")
    plt.scatter(point_2[0], point_2[1], c="blue")
    plt.plot([point_1[0], point_2[0]],[point_1[1], point_2[1]], c="gray")
    for obstacle_id, points in obstacles.items():
        start_x, start_y = points[0]
        end_x, end_y = points[1]
        plt.fill([start_x, end_x, end_x, start_x], [start_y, start_y, end_y, end_y],
                 label="Obstacle", alpha=0.7, color='red')
    plt.show()

In [3]:
def frange(start, stop, step):
    while start < stop:
        yield round(start, 4)
        start += step

In [4]:
def get_neighbors(point, grid):
  """
  Get valid neighboring grid cells for a given point.

  Args:
      point: (x, y) coordinates of the current point.
      grid: 2D grid representation of the environment.

  Returns:
      List of valid neighboring points (x, y).
  """
  neighbors = []
  x, y = point
  rows, cols = len(grid), len(grid[0])
  for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
    new_x, new_y = x + dx, y + dy
    if 0 <= new_x < rows and 0 <= new_y < cols and grid[new_x][new_y] != 1:
      neighbors.append((new_x, new_y))
  return neighbors

In [5]:
from collections import deque

def generate_path(point_1, point_2, obstacles, grid_resolution=1):
  # Define grid and mark obstacles
  grid = [[0 for _ in range(int((point_2[0] - point_1[0]) / grid_resolution) + 1)]
          for _ in range(int((point_2[1] - point_1[1]) / grid_resolution) + 1)]
  for obs_id,obstacle in obstacles.items():
    for i in frange(obstacle[0][0], obstacle[1][0] + 1, 0.2):
      for j in frange(obstacle[0][1], obstacle[1][1] + 1, 0.2):
        grid[i][j] = 1

  # Use A* search for example
  queue = deque([(point_1, 0)])  # (point, cost)
  visited = set()
  came_from = {}
  while queue:
    current, cost = queue.popleft()
    if current == point_2:
      break
    if current in visited:
      continue
    visited.add(current)
    for neighbor in get_neighbors(current, grid):
      if neighbor not in visited and grid[neighbor[0]][neighbor[1]] != 1:
        new_cost = cost + 1  # Adjust cost function if needed
        if neighbor not in queue or new_cost < queue[queue.index(neighbor)][1]:
          queue.append((neighbor, new_cost))
          came_from[neighbor] = current

  # Backtrack to reconstruct path
  path = []
  current = point_2
  while current != point_1:
    path.append(current)
    current = came_from[current]
  path.append(point_1)
  path.reverse()  # Optional: reverse for start -> end order

  return path

In [6]:
point_1 = [1,1]
point_2 = [5,5]

obstacles = {
    0 : [[2,2],[2.5,2.5]],
    1 : [[3,3],[3.5,3.5]]
}

path = generate_path(point_1, point_2, obstacles)

make_graph(point_1, point_2, obstacles)

TypeError: list indices must be integers or slices, not float